[Day 8 - Advent of Code 2023](https://adventofcode.com/2023/day/8)

<article class="day-desc">
 <h2>
  --- Day 8: Haunted Wasteland ---
 </h2>
 <p>
  You're still riding a camel across Desert Island when you spot a sandstorm quickly approaching. When you turn to warn the Elf, she disappears before your eyes! To be fair, she had just finished warning you about
  <em>
   ghosts
  </em>
  a few minutes ago.
 </p>
 <p>
  One of the camel's pouches is labeled "maps" - sure enough, it's full of documents (your puzzle input) about how to navigate the desert. At least, you're pretty sure that's what they are; one of the documents contains a list of left/right instructions, and the rest of the documents seem to describe some kind of
  <em>
   network
  </em>
  of labeled nodes.
 </p>
 <p>
  It seems like you're meant to use the
  <em>
   left/right
  </em>
  instructions to
  <em>
   navigate the network
  </em>
  . Perhaps if you have the camel follow the same instructions, you can escape the haunted wasteland!
 </p>
 <p>
  After examining the maps for a bit, two nodes stick out:
  <code>
   AAA
  </code>
  and
  <code>
   ZZZ
  </code>
  . You feel like
  <code>
   AAA
  </code>
  is where you are now, and you have to follow the left/right instructions until you reach
  <code>
   ZZZ
  </code>
  .
 </p>
 <p>
  This format defines each
  <em>
   node
  </em>
  of the network individually. For example:
 </p>
 <pre><code>RL

AAA = (BBB, CCC)
BBB = (DDD, EEE)
CCC = (ZZZ, GGG)
DDD = (DDD, DDD)
EEE = (EEE, EEE)
GGG = (GGG, GGG)
ZZZ = (ZZZ, ZZZ)
</code></pre>
 <p>
  Starting with
  <code>
   AAA
  </code>
  , you need to
  <em>
   look up the next element
  </em>
  based on the next left/right instruction in your input. In this example, start with
  <code>
   AAA
  </code>
  and go
  <em>
   right
  </em>
  (
  <code>
   R
  </code>
  ) by choosing the right element of
  <code>
   AAA
  </code>
  ,
  <code>
   <em>
    CCC
   </em>
  </code>
  . Then,
  <code>
   L
  </code>
  means to choose the
  <em>
   left
  </em>
  element of
  <code>
   CCC
  </code>
  ,
  <code>
   <em>
    ZZZ
   </em>
  </code>
  . By following the left/right instructions, you reach
  <code>
   ZZZ
  </code>
  in
  <code>
   <em>
    2
   </em>
  </code>
  steps.
 </p>
 <p>
  Of course, you might not find
  <code>
   ZZZ
  </code>
  right away. If you run out of left/right instructions, repeat the whole sequence of instructions as necessary:
  <code>
   RL
  </code>
  really means
  <code>
   RLRLRLRLRLRLRLRL...
  </code>
  and so on. For example, here is a situation that takes
  <code>
   <em>
    6
   </em>
  </code>
  steps to reach
  <code>
   ZZZ
  </code>
  :
 </p>
 <pre><code>LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)
</code></pre>
 <p>
  Starting at
  <code>
   AAA
  </code>
  , follow the left/right instructions.
  <em>
   How many steps are required to reach
   <code>
    ZZZ
   </code>
   ?
  </em>
 </p>
</article>


In [27]:
import helpers
from tqdm.notebook import tqdm
import numpy as np
import re
from itertools import cycle
from math import lcm

In [28]:
puzzle_input = helpers.import_input(8)

## Part 1

In [29]:
test_puzzle_input_part_1 = """RL

AAA = (BBB, CCC)
BBB = (DDD, EEE)
CCC = (ZZZ, GGG)
DDD = (DDD, DDD)
EEE = (EEE, EEE)
GGG = (GGG, GGG)
ZZZ = (ZZZ, ZZZ)
""".splitlines()

In [30]:
def part_1(input):
    solution = 0
    directions = cycle(input[0])
    rules = [x.split(' = ') for x in input[2:]]
    rules = [(x[0], x[1][1:-1].split(', ')) for x in rules]
    
    rules = dict(rules)
    currentPos = 'AAA'

    while currentPos != 'ZZZ':
        solution += 1
        direction = directions.__next__()

        if direction == 'L':
            currentPos = rules[currentPos][0]
        elif direction == 'R':
            currentPos = rules[currentPos][1]

    return solution
  

assert part_1(test_puzzle_input_part_1) == 2

In [31]:
part_1(puzzle_input)

19241

## Part 2

--- Part Two ---

The sandstorm is upon you and you aren't any closer to escaping the wasteland. You had the camel follow the instructions, but you've barely left your starting position. It's going to take significantly more steps to escape!

What if the map isn't for people - what if the map is for ghosts? Are ghosts even bound by the laws of spacetime? Only one way to find out.

After examining the maps a bit longer, your attention is drawn to a curious fact: the number of nodes with names ending in A is equal to the number ending in Z! If you were a ghost, you'd probably just start at every node that ends with A and follow all of the paths at the same time until they all simultaneously end up at nodes that end with Z.

In [32]:
test_puzzle_input_part_2 = """LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)
""".splitlines()

In [34]:
def part_2(input):
    rules = [x.split(' = ') for x in input[2:]]
    rules = [(x[0], x[1][1:-1].split(', ')) for x in rules]
    rules = {x[0]: {'L': x[1][0], 'R': x[1][1]} for x in rules}
    rules = dict(rules)
    
    currentPositions = [x for x in rules if x.endswith('A')]
    solutions = [0] * len(currentPositions)

    for i in range(len(currentPositions)):
        directions = cycle(input[0])
        while not currentPositions[i].endswith('Z'):
            solutions[i] += 1
            direction = directions.__next__()
            currentPositions[i] = rules[currentPositions[i]][direction]

    return lcm(*solutions)

assert part_2(test_puzzle_input_part_2) == 6

KeyboardInterrupt: 

In [ ]:
part_2(puzzle_input)